## 공동주택 관리비 확인하기

### "공동주택 단지 목록" 조회 URL
http://apis.data.go.kr/1611000/AptListService/getLegaldongAptList?loadCode=시군구법정동코드&_type=json&ServiceKey=인증키" 입니다

조회시, "광주시 경안동"과 같은 이름이 아니라   
시군구와 법정동을 합친 코드가 필요하다.  
이 코드를 "행정표준코드관리시스템"에서 가져올 수 있습니다.  
http://www.code.go.kr/jsp/index.jsp  
에서 법정동 클릭하고  
"법정동코드목록조회"화면에서  
지역선택은 경기도만 입력한 상태에서 "조회" 클릭  
"사용자검색자료"를 클릭  
파일 다운로드  
  -> 이렇게 받은 파일이 "생활데이터/apartment/kyunggi.txt"입니다.  
  
  

kyunggi.txt 파일 내용:  
법정동코드	법정동명  
4100000000	경기도  
4111000000	경기도 수원시  
4111100000	경기도 수원시 장안구  
4111112900	경기도 수원시 장안구 파장동  
4111113000	경기도 수원시 장안구 정자동  
4111113100	경기도 수원시 장안구 이목동  
4111113200	경기도 수원시 장안구 율전동  
4111113300	경기도 수원시 장안구 천천동  

파일의 한 라인씩 읽은 후, 해당동의 공동주택 관리비를 검색하려고 함  

사용 package :   
urllib.request에 urlopen - http를 통한 API 호출시 필요    
json                     - API결과를 해석하는데 사용    
csv                      - csv 파일을 읽는데 이용    

In [8]:
#-*- coding:utf-8 -*-
from urllib.request import urlopen
import json
import csv

baseURL1="http://apis.data.go.kr/1611000/AptListSevice/getLegaldongAptList?loadCode="
baseURL2="&_type=json&numOfRows=100&ServiceKey=6fC%2FE%2FOfXdNsSQPKEtWOgorYlxR7czIHUrRj8f6waylgRRJuo7qajzbZCz2Yn959nXz6bWMcDmS1vJG%2BJjk3tQ%3D%3D"

with open('kyunggi.txt', 'rt') as csvfile:
    rows=csv.reader(csvfile, delimiter='\t')
    print(rows)
    for r in rows:
        print(r[1])
        url=baseURL1+r[0]+baseURL2
        page=urlopen(url)
        
        response=page.read().decode('utf8')
        decoded=json.loads(response)
        print(decoded)


UnicodeDecodeError: 'cp949' codec can't decode byte 0xeb in position 6: illegal multibyte sequence

---------------------------------------------------------------------------
UnicodeDecodeError                        Traceback (most recent call last)
<ipython-input-8-6eba5631517b> in <module>
     10     rows=csv.reader(csvfile, delimiter='\t')
     11     print(rows)
---> 12     for r in rows:
     13         print(r[1])
     14         url=baseURL1+r[0]+baseURL2

UnicodeDecodeError: 'cp949' codec can't decode byte 0xeb in position 6: illegal multibyte sequence

    
encoding 에러가 납니다.
    
파이썬3부터는 ANSI기준으로 작성된 파일만 정상적으로 읽어 올수 있다고 합니다  
UTF-8로 작성된 파일은 그냥 읽으면 못 읽고 UnicodeDecodeError가 난다고 합니다.  
    
        with open('kyunggi.txt', 'rt') as csvfile:  
    
--->    with open('kyunggi.txt', 'rt', encoding='UTF-8') as csvfile:  

위와 같이 해주세요.

In [48]:
#-*- coding:utf-8 -*-
from urllib.request import urlopen
import json
import csv

baseURL1="http://apis.data.go.kr/1611000/AptListService/getLegaldongAptList?bjdCode="
baseURL2="&_type=json&numOfRows=100&ServiceKey=6fC%2FE%2FOfXdNsSQPKEtWOgorYlxR7czIHUrRj8f6waylgRRJuo7qajzbZCz2Yn959nXz6bWMcDmS1vJG%2BJjk3tQ%3D%3D"
with open('kaptcode.txt', 'w', encoding='UTF-8') as fw:
    with open('kyunggi.txt', 'rt', encoding='UTF-8') as csvfile:
        rows=csv.reader(csvfile, delimiter='\t')

        #header를 skip하기
        next(rows)

        print(rows)
        for r in rows:
            print(r[1])
            url=baseURL1+r[0]+baseURL2
            page=urlopen(url)

            response=page.read().decode('utf-8')
            decoded=json.loads(response)
            print(decoded)
            if decoded['response']['header']['resultCode'] != '00':
                print('query error')
                exit(1)

            total = decoded['response']['body']['totalCount']
            if total == 0 or decoded['response']['body']['items'] == '':
                continue
            elif isinstance(decoded['response']['body']['items']['item'], dict):
                print (decoded['response']['body']['items']['item']['kaptCode']+' '+decoded['response']['body']['items']['item']['kaptName'])
                ret=decoded['response']['body']['items']['item']['kaptCode']+' '+decoded['response']['body']['items']['item']['kaptName']
                fw.write(ret); fw.write('\n')
            elif isinstance(decoded['response']['body']['items']['item'], list):
                for item in decoded['response']['body']['items']['item']:
                    print(item['kaptCode']+' '+item['kaptName'])
                    ret=item['kaptCode']+' '+item['kaptName']
                    fw.write(ret); fw.write('\n')
            
        

경기도
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 수원시
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 수원시 장안구
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 수원시 장안구 파장동
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'kaptCode': 'A44085201', 'kaptName': '파장화남2차'}, {'kaptCode': 'A44085102', 'kaptName': '파장삼익'}, {'kaptCode': 'A44077501', 'kaptName': '북수원I-PARK'}, {'kaptCode': 'A44077502', 'kaptName': '파장현대아파트'}]}, 'numOfRows': 100, 'pageNo': 1, 'totalCount': 4}}}
A44085201 파장화남2차
A44085102 파장삼익
A44077501 북수원I-PARK
A44077502 파장현대아파트
경기도 수원시 장안구 정자동
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SE

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'kaptCode': 'A44176406', 'kaptName': '고색우림필유'}, {'kaptCode': 'A44180402', 'kaptName': '고색상우'}, {'kaptCode': 'A44172904', 'kaptName': '고색2차태산'}, {'kaptCode': 'A44172807', 'kaptName': '고색태산1차'}, {'kaptCode': 'A44172708', 'kaptName': '고색대한'}, {'kaptCode': 'A44172605', 'kaptName': '고색연합대원'}, {'kaptCode': 'A44172603', 'kaptName': '고색거산'}, {'kaptCode': 'A44172606', 'kaptName': '연안'}]}, 'numOfRows': 100, 'pageNo': 1, 'totalCount': 8}}}
A44176406 고색우림필유
A44180402 고색상우
A44172904 고색2차태산
A44172807 고색태산1차
A44172708 고색대한
A44172605 고색연합대원
A44172603 고색거산
A44172606 연안
경기도 수원시 권선구 오목천동
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'kaptCode': 'A44175602', 'kaptName': '오목천청구1차'}, {'kaptCode': 'A44176505', 'kaptName': '오목천대우푸르지오1단지'}, {'kaptCode': 'A44135004', 'kaptName': '오목천영조아름다운나날'}, {'kaptCode': 'A44184901', 'kaptName': '오목천태산'}, {'ka

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 수원시 권선구 대황교동
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 수원시 권선구 입북동
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'kaptCode': 'A44147003', 'kaptName': '서수원레이크푸르지오1단지'}, {'kaptCode': 'A44147002', 'kaptName': '서수원레이크푸르지오2단지'}, {'kaptCode': 'A44147001', 'kaptName': '서수원자이'}]}, 'numOfRows': 100, 'pageNo': 1, 'totalCount': 3}}}
A44147003 서수원레이크푸르지오1단지
A44147002 서수원레이크푸르지오2단지
A44147001 서수원자이
경기도 수원시 권선구 당수동
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'kaptCode': 'A44176102', 'kaptName': '당수한라비발디1단지'}, {'kaptCode': 'A44176104', 'kaptName': '당수한라비발디2단지'}, {'kaptCode': 'A44175203', 'kaptName': '당수인정프린스'}, {'kaptC

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'kaptCode': 'A44371004', 'kaptName': '매탄현대힐스테이트'}, {'kaptCode': 'A44380701', 'kaptName': '매탄금성'}, {'kaptCode': 'A44384808', 'kaptName': '매탄위브하늘채'}, {'kaptCode': 'A44371706', 'kaptName': '매탄원천성일'}, {'kaptCode': 'A44379502', 'kaptName': '매탄주공그린빌4단지'}, {'kaptCode': 'A44337202', 'kaptName': '매탄극동'}, {'kaptCode': 'A44379205', 'kaptName': '매탄주공그린빌1단지'}, {'kaptCode': 'A44380407', 'kaptName': '매탄임광'}, {'kaptCode': 'A44380201', 'kaptName': '매탄삼성3차'}, {'kaptCode': 'A44371608', 'kaptName': '매탄성일'}, {'kaptCode': 'A44371902', 'kaptName': '매탄삼성2차'}, {'kaptCode': 'A44379709', 'kaptName': '매탄주공그린빌6단지'}, {'kaptCode': 'A44370501', 'kaptName': '매탄이편한세상'}, {'kaptCode': 'A44379606', 'kaptName': '매탄주공그린빌5단지'}, {'kaptCode': 'A44380103', 'kaptName': '매탄동남빌라'}, {'kaptCode': 'A44371303', 'kaptName': '매탄현대'}, {'kaptCode': 'A44370903', 'kaptName': '매탄주공5단지'}, {'kaptCode': 'A44372005', 'kaptName': '한국2차'}, {'

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': {'item': [{'kaptCode': 'A44370614', 'kaptName': '동수원자이1차'}, {'kaptCode': 'A44370703', 'kaptName': '센트럴하이츠'}, {'kaptCode': 'A44340001', 'kaptName': '영통삼정라츠'}, {'kaptCode': 'A44376911', 'kaptName': '동수원엘지빌리지2차'}, {'kaptCode': 'A44377707', 'kaptName': '망포2차현대아이파크'}, {'kaptCode': 'A44376808', 'kaptName': '망포역마을쌍용1차아파트'}, {'kaptCode': 'A44340013', 'kaptName': '망포늘푸른벽산'}, {'kaptCode': 'A44370704', 'kaptName': '한양수자인에듀파크'}, {'kaptCode': 'A44374606', 'kaptName': '벽산e빌리지 아파트'}, {'kaptCode': 'A44340005', 'kaptName': '동수원3차쌍용스윗닷홈'}, {'kaptCode': 'A44370701', 'kaptName': '그대가센트럴파크'}, {'kaptCode': 'A44377009', 'kaptName': '망포1차현대아이파크'}, {'kaptCode': 'A44370702', 'kaptName': '그대가프리미어'}, {'kaptCode': 'A44374510', 'kaptName': '영통뜨란채'}, {'kaptCode': 'A44340012', 'kaptName': '엘지동수원자이'}, {'kaptCode': 'A44340003', 'kaptName': '영통마젤란21'}, {'kaptCode': 'A44340004', 'kaptName': '동수원2차쌍용스윗닷홈'}, {'kaptCode': 'A44370

...

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 군남면 진상리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 군남면 옥계리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 군남면 삼거리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 군남면 왕림리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 군남면 선곡리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 군남면 황지리
{'re

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 왕징면 고왕리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 왕징면 고잔상리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 왕징면 고잔하리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 왕징면 기곡리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 왕징면 작동리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 연천군 왕징면 임강리
{'

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 가평읍 두밀리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 가평읍 경반리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 가평읍 승안리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 가평읍 마장리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 가평읍 개곡리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 가평읍 이화리
{'re

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 조종면 신하리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 조종면 마일리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 조종면 대보리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 조종면 운악리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 북면
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 가평군 북면 목동리
{'response

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 양평군 양동면 금왕리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 양평군 양동면 계정리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 양평군 양동면 삼산리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 양평군 양동면 단석리
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 양평군 지평면
{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}, 'body': {'items': '', 'numOfRows': 100, 'pageNo': 1, 'totalCount': 0}}}
경기도 양평군 지평면 지평리
{'respon

# 아파트 수리비 구하기

In [23]:
from urllib.request import urlopen
import json
import csv

countRpr=0
countHo=0
sum=0
countAll=0

repairURL1 = 'http://apis.data.go.kr/1611000/AptRepairsCostService/getHsmpMonthRetalFeeInfo?kaptCode='
repairURL2='&_type=json&searchDate=202001&ServiceKey=6fC%2FE%2FOfXdNsSQPKEtWOgorYlxR7czIHUrRj8f6waylgRRJuo7qajzbZCz2Yn959nXz6bWMcDmS1vJG%2BJjk3tQ%3D%3D'

infoURL1='http://apis.data.go.kr/1611000/AptBasisInfoService/getAphusBassInfo?kaptCode='
infoURL2='&_type=json&ServiceKey=6fC%2FE%2FOfXdNsSQPKEtWOgorYlxR7czIHUrRj8f6waylgRRJuo7qajzbZCz2Yn959nXz6bWMcDmS1vJG%2BJjk3tQ%3D%3D'

with open('kaptcode.txt','rt', encoding='utf-8') as csvfile:
    rows=csv.reader(csvfile, delimiter=' ')
    for r in rows:
        print(r)
        if r[0][0] != 'A':
            continue
        url=repairURL1+r[0]+repairURL2
        print(url)
        page=urlopen(url)
        print(page)
        response=page.read().decode('utf-8')
        print(response)
        decoded=json.loads(response)
        print(decoded)
        try:
            repair=decoded['response']['body']['item']['SUse']
        except TypeError:
            countRpr +=1
            continue
        url=infoURL1+r[0]+infoURL2
        page=urlopen(url)
        response=page.read().decode('utf-8')
        decoded=json.loads(response)
        print(decoded)
        try:
            ho=decoded['response']['body']['item']['kaptdaCnt']
        except TypeError:
            countHo += 1
            continue
        repair=10000
        repairPer = int(float(repair)/float(ho))
        sum += repairPer
        countAll += 1
        print(decoded['response']['body']['item']['kaptName'], repairPer)
        
print('Summary')
print('# of missing repair', countRpr)
print('# of missing hoCnt', countHo)
print('# of valid repair and hoCnt', countAll)
print('average repair Cost', sum/countAll)


['A44085201', '파장화남2차']
http://apis.data.go.kr/1611000/AptRepairsCostService/getHsmpMonthRetalFeeInfo?kaptCode=A44085201&_type=json&searchDate=202001&ServiceKey=6fC%2FE%2FOfXdNsSQPKEtWOgorYlxR7czIHUrRj8f6waylgRRJuo7qajzbZCz2Yn959nXz6bWMcDmS1vJG%2BJjk3tQ%3D%3D


HTTPError: HTTP Error 500: Internal Server Error